Мария Сухарева

В моем тексте я выделила несколько сложных моментов. Мой корпус начинается со слова "собес", что является аббревиатурой, я его разметила как PROPN по выбранному тегсету, но из-за того, что в русском языке это слово уже не пишут прописными буквами, некоторые теггеры разметили его как просто существительное. Далее я включила два разных слова с дефисом внутри, чтобы посмотреть, как разные теггеры будут их делить, слово "из-за" делить не надо было, а "диван-кровать" - надо, один теггер делил все, другой - ничего, но один все-таки справился. Также я включила в свой корпус такое словосочетание с омонимией, как "маша руками", где "маша" нужно разметить как глагол, но большинство теггеров его размечают, как имя собственное. Еще я включила два непростых слова "меццо" и "сопрано", не все теггеры разметили это как существительные. Кроме того, можно встретить три субстантивированных существительных - "скорые", "раненые" и "погибшие", не все теггеры разметили их как существительные. Еще один интересный момент - у меня было слово "вжжик", которое надо разметить как INTJ, но большинство теггеров размечают его как-то по-другому. Еще в моем корпусе есть слово "Танину", что хотелось бы разметить как ADJ, потому что это все-таки притяжательное прилагательное, но некоторые теггеры размечали его как имя собственное. И также было несколько моментов, где можно было перепутать наречие это или краткое прилагательное и, наоборот, с чем тоже справились не все теггеры.

В моей разметке я использовала тегсет Universal POS tags, потому что это один из самых распространенных тегсетов, его используют многие различные POS теггеры. В моем корпусе я не ставила задачу различать разные виды существительных (просто существительные и местоимения-существительные, например), прилагательных (числительное-прилагательное, местоимение-прилагательное, например) и так далее, а мне было более интересно помотреть на то, как теггеры размечают глобальные части речи, если можно так сказать, то есть не вводить что-то смешанное, а смотреть на все более раздельно. Кроме того, в Universal POS tags есть различие между сочинительными и подчинительными союзами (CCONJ и SCONJ), что как раз хорошо подходит для русского языка. Я не стала выделять отдельно причастия и деепричастия, потому что это можно посмотреть в дополнительных признаках, и на самом деле многие теггеры используют тегсеты, где это отдельно не выделяется. Еще мне показалось хорошим то, что в Universal POS tags отдельно выделяется AUX, что хорошо для русского "быть" в некоторых конструкциях.

In [1]:
#загружаю свой размеченный корпус и делю его на два списка: один с токенами, другой с POS тегами
import pandas as pd

text = open('changedtime.txt', encoding='utf-8').read()

token_mine = []
pos_mine = []

df = pd.read_excel('changedtime_tags.xlsx')
for i in range(df.shape[0]):
    token_mine.append(df.iloc[i]['token'])
    pos_mine.append(df.iloc[i]['pos'])

In [2]:
#прогоняю текст через первый теггер, результат также делю на два списка
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

doc = Doc(text)
doc.segment(segmenter)
doc.tag_morph(morph_tagger)

token_natasha = []
pos_natasha = []

for _ in doc.tokens:
    token_natasha.append(_.text)
    pos_natasha.append(_.pos)

In [3]:
#аналогично для второго и третьего
import spacy

nlp = spacy.load('ru_core_news_md')
document = nlp(text)

token_spacy = []
pos_spacy = []

for token in document:
    token_spacy.append(str(token))
    pos_spacy.append(token.pos_)

In [4]:
import stanza

nlps = stanza.Pipeline(lang='ru', processors='tokenize,pos')
doc = nlps(text)

token_stanza = []
pos_stanza = []

for sent in doc.sentences:
    for word in sent.words:
        token_stanza.append(word.text)
        pos_stanza.append(word.upos)

2022-10-11 22:43:12 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2022-10-11 22:43:13 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |

2022-10-11 22:43:13 INFO: Use device: cpu
2022-10-11 22:43:13 INFO: Loading: tokenize
2022-10-11 22:43:13 INFO: Loading: pos
2022-10-11 22:43:13 INFO: Done loading processors!


Мне не понадобилась функция-конвертор для POS тегов, потому что все выбранные мной библиотеки используют тегсет Universal POS tags.

Далее идет моя функция, которая записывает в результаты единицу, если POS теги совпали, и ноль, если нет. В этой функции я иду по спискам токенов - моего эталона и сравниваемого теггера. Возможно, списки в данном случае - не очень хорошо, можно придумать более эффективный способ, но это все, что пришло мне в голову. Я думала использовать словари, но тогда не понятно, что брать за ключи, так как токены брать нельзя, они могут повторяться. Далее я обращаюсь по индексу к списку POS тегов, индексы как раз будут равны, если токены совпадают. Я ввела дополнительные смещения индекса bias, так как не все теггеры правильно делят слова с дефисами. Там, где я проверяю, равен ли токен конкретно "из-за" или "диван-кровати", хорошо бы заменить на что-то более общее, но я не смогла придумать, как это можно развести получше, а так как мой корпус небольшой, и я знала, что там всего два таких конкретных слова, то я просто проверяла по токену, что, конечно, к сожалению, совсем неэффективно. 

In [5]:
def compare(token, pos, token_mine, pos_mine):
    
    bias_mine = 0
    bias = 0
    results = []
    
    if token > token_mine:
        tokens = token
    else:
        tokens = token_mine
        
    for i in range(len(tokens)):
        if token[i+bias] == token_mine[i+bias_mine]:
            if pos[i+bias] == pos_mine[i+bias_mine]:
                results.append(1)
            else:
                results.append(0)
        elif token_mine[i+bias_mine] == 'из-за':
            results.append(0)
            bias += 2
        elif token[i+bias] == 'диван-кровати':
            results.append(0)
            bias_mine += 2
        else:
            results.append(0)
            
    return results

In [6]:
#оцениваю accuracy
from sklearn.metrics import accuracy_score

In [7]:
#для natasha
results_natasha = compare(token_natasha, pos_natasha, token_mine, pos_mine)
#для моих результатов я создаю список из единичек длиной, как список оцениваемого теггера, что скорее всего тоже не совсем эффективно, но я не смогла придумать лучше
results_mine = []

for i in range(len(results_natasha)):
    results_mine.append(1)

accuracy_natasha = accuracy_score(results_natasha, results_mine)
accuracy_natasha

0.9007633587786259

In [8]:
#для spacy
results_spacy = compare(token_spacy, pos_spacy, token_mine, pos_mine)

results_mine = []

for i in range(len(results_spacy)):
    results_mine.append(1)

accuracy_spacy = accuracy_score(results_spacy, results_mine)
accuracy_spacy

0.8977272727272727

In [9]:
#для stanza
results_stanza = compare(token_stanza, pos_stanza, token_mine, pos_mine)

results_mine = []

for i in range(len(results_stanza)):
    results_mine.append(1)

accuracy_stanza = accuracy_score(results_stanza, results_mine)
accuracy_stanza

0.9242424242424242

Лучшим оказался stanza.

Далее я создаю функцию (chunker) для пяти типов n-грамм:
1. ADJ + NOUN - достаточно очевидная биграмма для определения положительных и отрицательных отзывов, потому что в отзывах часто встречается описание какого-то объекта положительными или отрицательными прилагательными (хороший бассейн, плхая кровать)
2. ADV + VERB - такие сочетания тоже достаточно часто встречаются в отзывах, особенно, если помнить, что в stanza VERB может быть не только просто глаголом, но и еще причастием и деепричастием (плохо/хорошо сделан)
4. "не" + VERB - может встретиться в отрицательных отзывах (не работает, не положили)

In [10]:
def chunker(pos, token):
    
    chunks = []
    
    for i in range(len(pos)):
        if pos[i] == 'ADJ' and pos[i+1] == 'NOUN':
            chunks.append(token[i]+' '+token[i+1])
        if pos[i] == 'ADV' and pos[i+1] == 'VERB':
            chunks.append(token[i]+' '+token[i+1])
        if token[i] == 'не' and pos[i+1] == 'VERB':
            chunks.append(token[i]+' '+token[i+1])
            
    return chunks